# 8 機械学習の基礎

In [1]:
# 途中で使用するため、あらかじめ読み込んでおいてください。
# データ加工・処理・分析モジュール
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series,DataFrame
import pandas as pd

# 可視化モジュール
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# 機械学習モジュール
import sklearn

# 小数第３まで表示
%precision 3

'%.3f'

## 8.2 教師あり学習

## 8.2.1 重回帰分析

In [2]:
# 自動車価格データの取得
import requests, zipfile
from io import StringIO
import io

# url 
auto_data_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data"
s = requests.get(auto_data_url).content
auto_data = pd.read_csv(io.StringIO(s.decode('utf-8')),header=None)
auto_data.columns =["symboling","normalized-losses","make","fuel-type"
                     ,"aspiration","num-of-doors","body-style","drive-wheels","engine-location","wheel-base","length"
                   ,"width","height","curb-weight","engine-type","num-of-cylinders","engine-size","fuel-system"
                    ,"bore","stroke","compression-ratio","horsepower","peak-rpm","city-mpg","highway-mpg","price"]

In [6]:
for col_name in auto_data.columns:
    print(col_name,sum(auto_data[col_name].isin(['?'])))

symboling 0
normalized-losses 41
make 0
fuel-type 0
aspiration 0
num-of-doors 2
body-style 0
drive-wheels 0
engine-location 0
wheel-base 0
length 0
width 0
height 0
curb-weight 0
engine-type 0
num-of-cylinders 0
engine-size 0
fuel-system 0
bore 4
stroke 4
compression-ratio 0
horsepower 2
peak-rpm 2
city-mpg 0
highway-mpg 0
price 4


In [11]:
sub_auto_data = auto_data[['price','horsepower','width', 'height']]
sub_auto_data = sub_auto_data.replace('?', np.nan).dropna()
sub_auto_data['price'] = pd.to_numeric(sub_auto_data['price'])
sub_auto_data['horsepower'] = pd.to_numeric(sub_auto_data['horsepower'])

In [12]:
sub_auto_data.head()

,price,horsepower,width,height
0,13495,111,64.1,48.8
1,16500,111,64.1,48.8
2,16500,154,65.5,52.4
3,13950,102,66.2,54.3
4,17450,115,66.4,54.3


In [13]:
sub_auto_data.corr()

,price,horsepower,width,height
price,1.000000,0.810533,0.753871,0.134990
horsepower,0.810533,1.000000,0.615315,-0.087407
width,0.753871,0.615315,1.000000,0.309223
height,0.134990,-0.087407,0.309223,1.000000


In [27]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
l_model = linear_model.LinearRegression()
X = sub_auto_data.drop("price", axis=1)
Y = sub_auto_data["price"]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=0)
clf = l_model.fit(X_train,y_train)
print("train:", clf.__class__.__name__, clf.score(X_train, y_train))
print("test:", clf.__class__.__name__, clf.score(X_test, y_test))

print(pd.DataFrame({"Name":X.columns, "Coefficients":clf.coef_}).sort_values(by="Coefficients"))
print("Intercept:", clf.intercept_)

train: LinearRegression 0.73335756839
test: LinearRegression 0.737068873813
   Coefficients        Name
0     81.651078  horsepower
2    229.510077      height
1   1829.174506       width
Intercept: -128409.046303


#### 練習問題1

In [35]:
pr1_data = auto_data[['price', 'length', 'engine-size']]
pr1_data = pr1_data.replace('?', np.nan).dropna()
pr1_data['price'] = pd.to_numeric(pr1_data['price'])
Y = pr1_data['price']
X = pr1_data.drop('price', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=0)
l_model = linear_model.LinearRegression()
clf = l_model.fit(X_train, y_train)
print('train:', clf.__class__.__name__, clf.score(X_train, y_train))
print('test:', clf.__class__.__name__, clf.score(X_test, y_test))
print(pd.DataFrame({'Name':X.columns, 'Coefficients':clf.coef_}).sort_values(by='Coefficients'))
print('Intercept:', clf.intercept_)

train: LinearRegression 0.770736371407
test: LinearRegression 0.763418675554
   Coefficients         Name
1    120.273905  engine-size
0    179.544547       length
Intercept: -33590.4109916


### 8.2.2 ロジスティック回帰

In [39]:
# データの読み込み
adult_data_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
s=requests.get(adult_data_url).content
adult_data = pd.read_csv(io.StringIO(s.decode('utf-8')),header=None)
adult_data.columns =["age","workclass","fnlwgt","education"
                     ,"education-num","marital-status","occupation"
                     ,"relationship","race","sex","capital-gain"
                     ,"capital-loss","hours-per-week","native-country","flg-50K"]

In [40]:
adult_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education-num     32561 non-null int64
marital-status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital-gain      32561 non-null int64
capital-loss      32561 non-null int64
hours-per-week    32561 non-null int64
native-country    32561 non-null object
flg-50K           32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [62]:
adult_data.groupby("flg-50K").size()

flg-50K
 <=50K    24720
 >50K      7841
dtype: int64

In [68]:
adult_data["fin_flg"] = [1 if i==" <=50K" else 0 for i in adult_data["flg-50K"]]

In [69]:
adult_data.groupby("fin_flg").size()

fin_flg
0     7841
1    24720
dtype: int64

In [77]:
# ロジスティック回帰
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = adult_data[["age","fnlwgt","education-num","capital-gain","capital-loss"]]
Y = adult_data["fin_flg"]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=0)

# 標準化
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

l_model= linear_model.LogisticRegression()
clf = l_model.fit(X_train_std, y_train)
print("train;", clf.__class__.__name__, clf.score(X_train_std, y_train))
print("test:", clf.__class__.__name__, clf.score(X_test_std, y_test))
print(clf.coef_)

train; LogisticRegression 0.810565110565
test: LogisticRegression 0.81014679688
[[-0.533 -0.025 -0.839 -2.287 -0.287]]


In [91]:
from sklearn import linear_model

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
X = cancer.data
Y = cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=0, stratify = cancer.target)

l_model = linear_model.LinearRegression()
r_model = linear_model.Ridge()

for model in [l_model, r_model]:
    clf = model.fit(X_train, y_train)
    print("train:", clf.__class__.__name__, clf.score(X_train, y_train))
    print("test:", clf.__class__.__name__, clf.score(X_test, y_test))

train: LinearRegression 0.781333144845
test: LinearRegression 0.749935055996
train: Ridge 0.74952090761
test: Ridge 0.720967160862
